## 유흥업소

In [24]:
from geoband.API import *
import pandas as pd
import folium
import json
import geopandas as gpd

import re
import numpy as np

import datetime
from pyproj import Geod
from shapely import wkt
from shapely.geometry import *
from fiona.crs import *

In [25]:
# 작업한 파일 로컬로 다운로드 함수
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [26]:
police_border = gpd.read_file('./data/1.5개시도_경찰서_관할경계.geojson')
police_border.head(3)

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."


In [27]:
pub_con = pd.read_csv('./data/21.5개시도_유흥업소_단란주점현황.csv')
pub_con.head(3)

,license_date,open_gbn,type,surroundings,lon,lat
0,19951012,영업/정상,단란주점,NaN,127.300127,36.601953
1,20181127,영업/정상,단란주점,NaN,127.301023,36.601199
2,20021121,영업/정상,단란주점,유흥업소밀집지역,127.300940,36.601856


In [28]:
from tqdm import tqdm
tqdm.pandas()

In [29]:
# csv to geopandas
# lon, lat data를 geometry로 변경
pub_con['lat'] = pub_con['lat'].astype(float)
pub_con['lon'] = pub_con['lon'].astype(float)
pub_con['geometry'] = pub_con.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
pub_con = gpd.GeoDataFrame(pub_con, geometry='geometry')
pub_con.crs = {'init':'epsg:4326'}
pub_con = pub_con.to_crs({'init':'epsg:4326'}) # 좌표계 epsg : 4326

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [30]:
#경찰서랑 이어주기 
def detect_police_name(geometry) :
    for i in range(len(police_border['NAME'].unique())) :    # 보안등 위치의 geometry 와 관할서의 geometry를 비교하여
        if geometry.within(police_border.iloc[i].geometry) :             #  매칭된 관할서명을 return하는 함수
            return police_border.iloc[i].NAME         

In [31]:
pub_con['jur_stn'] = '' # 함수 적용
pub_con['jur_stn'] = pub_con.progress_apply(lambda x : detect_police_name(x['geometry']) , axis=1 )

100%|██████████| 7119/7119 [01:51<00:00, 64.06it/s] 


In [32]:
pub_con

,license_date,open_gbn,type,surroundings,lon,lat,geometry,jur_stn
0,19951012,영업/정상,단란주점,NaN,127.300127,36.601953,POINT (127.30013 36.60195),세종경찰서
1,20181127,영업/정상,단란주점,NaN,127.301023,36.601199,POINT (127.30102 36.60120),세종경찰서
2,20021121,영업/정상,단란주점,유흥업소밀집지역,127.300940,36.601856,POINT (127.30094 36.60186),세종경찰서
3,20020409,영업/정상,단란주점,유흥업소밀집지역,127.299671,36.602782,POINT (127.29967 36.60278),세종경찰서
4,19990804,영업/정상,단란주점,학교정화(상대),127.205025,36.678746,POINT (127.20503 36.67875),세종경찰서
...,...,...,...,...,...,...,...,...
7114,20001116,영업/정상,룸살롱,유흥업소밀집지역,126.930536,37.610393,POINT (126.93054 37.61039),서울은평경찰서
7115,20010307,영업/정상,룸살롱,유흥업소밀집지역,126.930733,37.610163,POINT (126.93073 37.61016),서울은평경찰서
7116,20170630,영업/정상,룸살롱,유흥업소밀집지역,126.929299,37.610537,POINT (126.92930 37.61054),서울은평경찰서
7117,20171228,영업/정상,룸살롱,유흥업소밀집지역,126.919550,37.620091,POINT (126.91955 37.62009),서울은평경찰서


In [33]:
#surrounding은 고려안하기로 했으니 삭제
pub_con = pub_con.drop(['surroundings'], axis=1)

In [34]:
#데이터에 나온 모든 지점들이 정상영업을 하기 때문에 삭제 
pub_con = pub_con.drop(['open_gbn'],axis=1)

In [40]:
# 연도 상,하반기로 나눠주기
condition = [(pub_con['license_date'] < 20170101),
            (pub_con['license_date'] >= 20170101) & (pub_con['license_date'] < 20170701), #2017상반기
            (pub_con['license_date'] >= 20170701) & (pub_con['license_date'] < 20180101), #2017하반기
            (pub_con['license_date'] >= 20180101) & (pub_con['license_date'] < 20180701), #2018상반기
            (pub_con['license_date'] >= 20180701) & (pub_con['license_date'] < 20190101), #2018하반기
            (pub_con['license_date'] >= 20190101) & (pub_con['license_date'] < 20190701), #2019상반기
            (pub_con['license_date'] >= 20190701) & (pub_con['license_date'] < 20200101), #2019하반기
            (pub_con['license_date'] >= 20200101) & (pub_con['license_date'] < 20200701), #2020상반기             
            (pub_con['license_date'] >= 20200701) & (pub_con['license_date'] < 20210101), #2020하반기
            (pub_con['license_date'] >= 20210101) & (pub_con['license_date'] < 20210701), #2021상반기             
            (pub_con['license_date'] >= 20210701) & (pub_con['license_date'] < 20220101), #2021하반기
            ]
result = ['2016(이전)', '2017(상반기)', '2017(하반기)','2018(상반기)','2018(하반기)','2019(상반기)',
          '2019(하반기)','2020(상반기)','2020(하반기)','2021(상반기)','2021(하반기)']
pub_con['year'] = np.select(condition, result, default = '')

In [41]:
pub_con

,license_date,type,lon,lat,geometry,jur_stn,year
0,19951012,단란주점,127.300127,36.601953,POINT (127.30013 36.60195),세종경찰서,2016(이전)
1,20181127,단란주점,127.301023,36.601199,POINT (127.30102 36.60120),세종경찰서,2018(하반기)
2,20021121,단란주점,127.300940,36.601856,POINT (127.30094 36.60186),세종경찰서,2016(이전)
3,20020409,단란주점,127.299671,36.602782,POINT (127.29967 36.60278),세종경찰서,2016(이전)
4,19990804,단란주점,127.205025,36.678746,POINT (127.20503 36.67875),세종경찰서,2016(이전)
...,...,...,...,...,...,...,...
7114,20001116,룸살롱,126.930536,37.610393,POINT (126.93054 37.61039),서울은평경찰서,2016(이전)
7115,20010307,룸살롱,126.930733,37.610163,POINT (126.93073 37.61016),서울은평경찰서,2016(이전)
7116,20170630,룸살롱,126.929299,37.610537,POINT (126.92930 37.61054),서울은평경찰서,2017(상반기)
7117,20171228,룸살롱,126.919550,37.620091,POINT (126.91955 37.62009),서울은평경찰서,2017(하반기)


In [17]:
pub_con = pub_con.drop(['lon','lat'],axis=1)

In [44]:
pub_con_cnt = pub_con[['jur_stn','year','type','geometry']]
pub_con_cnt

,jur_stn,year,type,geometry
0,세종경찰서,2016(이전),단란주점,POINT (127.30013 36.60195)
1,세종경찰서,2018(하반기),단란주점,POINT (127.30102 36.60120)
2,세종경찰서,2016(이전),단란주점,POINT (127.30094 36.60186)
3,세종경찰서,2016(이전),단란주점,POINT (127.29967 36.60278)
4,세종경찰서,2016(이전),단란주점,POINT (127.20503 36.67875)
...,...,...,...,...
7114,서울은평경찰서,2016(이전),룸살롱,POINT (126.93054 37.61039)
7115,서울은평경찰서,2016(이전),룸살롱,POINT (126.93073 37.61016)
7116,서울은평경찰서,2017(상반기),룸살롱,POINT (126.92930 37.61054)
7117,서울은평경찰서,2017(하반기),룸살롱,POINT (126.91955 37.62009)


In [45]:
pub_con_cnt.to_csv('./21번.유흥업소_processed.csv')

In [48]:
csv_download_link(pub_con_cnt, '21번.유흥업소_processed.csv')

/opt/app-root/src/21번.유흥업소_processed.csv

Press enter to delete the file after you have downloaded it.
